In [57]:
import pandas as pd
import numpy as np
import threading
from geopy.geocoders import Nominatim

In [10]:
geopoints = pd.read_csv("geopoints.csv")
geopoints

,latitude,longitude
0,-3.732950,-38.488054
1,-3.733045,-38.487666
2,-3.727074,-38.488931
3,-3.732315,-38.487305
4,-3.732324,-38.487482
...,...,...
6368,-3.751305,-38.529537
6369,-3.768745,-38.570932
6370,-3.769024,-38.570521
6371,-3.778014,-38.482875


In [13]:
geopoints_tuples = list(geopoints.itertuples(index=False, name=None))
geopoints_tuples

[(-3.732950297999232, -38.48805449308904),
 (-3.7330451606149113, -38.48766648645134),
 (-3.7270743803449737, -38.488930632436414),
 (-3.732315277120804, -38.48730511839091),
 (-3.732323732830626, -38.48748189341989),
 (-3.7323299740172033, -38.48766649022929),
 (-3.732346375135369, -38.48808676320801),
 (-3.732793894890859, -38.4844937982232),
 (-3.800728522318616, -38.60186390612826),
 (-3.799456986611557, -38.6054564149804),
 (-3.801328169589413, -38.60075309724794),
 (-3.733141802469878, -38.48208888561246),
 (-3.741667472411673, -38.48012351140471),
 (-3.7350228043200335, -38.48290662105191),
 (-3.7446264444536985, -38.47065123181685),
 (-3.744576091745832, -38.47105953987306),
 (-3.744817941852248, -38.47016104849294),
 (-3.727850657481922, -38.46368033807048),
 (-3.728166504147733, -38.46369004892053),
 (-3.732224001099402, -38.46171850513163),
 (-3.7492517912132297, -38.45405432635804),
 (-3.745932157603729, -38.48311666370157),
 (-3.746203988357183, -38.48855974630988),
 (-3.7

In [17]:
geopoints_array = np.array(geopoints_tuples, dtype="f,f")
geopoints_array

array([(-3.7329502, -38.488056), (-3.733045 , -38.487667),
       (-3.7270744, -38.48893 ), ..., (-3.7690241, -38.570522),
       (-3.7780142, -38.482876), (-3.7376056, -38.47989 )],
      shape=(6373,), dtype=[('f0', '<f4'), ('f1', '<f4')])

In [53]:
partition_1 = np.split(geopoints_array[0:6000], 4)

In [56]:
partition_1[0]

array([(-3.7329502, -38.488056), (-3.733045 , -38.487667),
       (-3.7270744, -38.48893 ), ..., (-3.7332122, -38.56867 ),
       (-3.731856 , -38.55587 ), (-3.7575512, -38.550602)],
      shape=(1500,), dtype=[('f0', '<f4'), ('f1', '<f4')])

In [ ]:
class BairrosThread(threading.Thread):
    def __init__(self, lista):
        super().__init__()
        self.lista = lista

    def run(self):
        geolocator = Nominatim(user_agent="geolocate_bairro")

        bairros = []
        subconjunto_bairros = []
        counter = 0

        for point in self.lista:
        # for point in somePoints:

            # print(point)
            
            location = geolocator.reverse(f"{point[0]}, {point[1]}", exactly_one=True, timeout=10)                                                                                                          #pois o outro sistema UTM é longitude, latitude

            if location:
                address = location.raw.get('address', {})
                bairro = address.get("neighbourhood") or address.get("suburb") or address.get("district")
                subconjunto_bairros.append(bairro)
            else:
                print("Localização não encontrada")

            counter = counter + 1
            if counter%100 == 0:
                bairros.append(subconjunto_bairros)
                print(subconjunto_bairros)
                subconjunto_bairros = []
        


threads = [BairrosThread(geopoints) for geopoints in partition_1]
for thread in threads:
    thread.start()

# Aguardando todos os threads terminarem
for thread in threads:
    thread.join()

print("Todos as threads foram concluídas!")

['Varjota', 'Varjota', 'Mucuripe', 'Varjota', 'Varjota', 'Varjota', 'Varjota', 'Varjota', 'Bom Jardim', 'Siqueira', 'Bom Jardim', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Vicente Pinzón', 'Vicente Pinzón', 'Vicente Pinzón', 'Praia do Futuro II', 'Cocó', 'Cocó', 'Engenheiro Luciano Cavalcante', 'Engenheiro Luciano Cavalcante', 'Conjunto Santa Luzia do Cocó', 'Engenheiro Luciano Cavalcante', 'Guararapes', 'Guararapes', 'Guararapes', 'São João do Tauape', 'Guararapes', 'Conjunto Santa Luzia do Cocó', 'Edson Queiroz', 'Edson Queiroz', 'Edson Queiroz', 'Sapiranga / Coité', 'Cambeba', 'Cambeba', 'Boa Vista/Castelão', 'Messejana', 'Messejana', 'Jardim Violeta', 'Cocó', 'De Lourdes', 'Praia de Iracema', 'Carlito Pamplona', 'Cocó', 'Meireles', 'Centro', 'Meireles', 'Aldeota', 'Meireles', 'Dias Macêdo', 'Carlito Pamplona', 'Mucuripe', 'Meireles', 'Carlito Pamplona', 'Centro', 'Praia do Futuro I', 'Meireles', 'Bela Vista', 'Engenheiro Luciano Cavalcante', 'São João do Tauape',